## Capstone Project – The Battle of Neighborhoods | Finding a Better Place To open a veterinary in Bogota-Colombia

### 1. Installing and Importing Python Libraries and Dependencies

In [1]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [6]:
## pip install folium
pip install geopy

     |████████████████████████████████| 112kB 6.6MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Import libraries

In [1]:
import pandas as pd
import requests
import numpy as np
import geocoder
##import folium
import requests 
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
import xml
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

from pandas.io.json import json_normalize 
from sklearn.cluster import KMeans
## from geopy.geocoders import Nominatim 
from bs4 import BeautifulSoup

import folium # map rendering library
from geopy.geocoders import Nominatim 


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

print("All Required Libraries Imported!")

All Required Libraries Imported!


### 2. Data Extraction and Cleaning

In [2]:
# @hiddel_cell
CLIENT_ID = 'IHMMGHNEYAR1IDVX4C4L2MJGH0FXBAGGJYAGRITQVVS2OPAB' # my Foursquare ID
CLIENT_SECRET = '4YLFZ2NWCQO1XGAVP2MTQ2LCNAHZDADTJBS0FDI4HY4SUJNZ' # my Foursquare Secret
VERSION = '20180604'
LIMIT = 300
print('Your credentails:')
print('CLIENT_ID: '+CLIENT_ID)
print('CLIENT_SECRET: '+CLIENT_SECRET)

Your credentails:
CLIENT_ID: IHMMGHNEYAR1IDVX4C4L2MJGH0FXBAGGJYAGRITQVVS2OPAB
CLIENT_SECRET: 4YLFZ2NWCQO1XGAVP2MTQ2LCNAHZDADTJBS0FDI4HY4SUJNZ


In [77]:
LIMIT = 100
latitude_n1= 4.710113
longitude_n1= -74.072036
query = 'veterinaria'
radius = 10000
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude_n1, 
   longitude_n1,
    query,
    radius, 
   LIMIT)
results = requests.get(url).json()

In [78]:
## veterinary=results['response']['groups'][0]['items']
veterinary=results['response']['venues']
veterinary = json_normalize(veterinary)
veterinary.columns

Index(['id', 'name', 'categories', 'referralId', 'hasPerk', 'location.lat',
       'location.lng', 'location.labeledLatLngs', 'location.distance',
       'location.cc', 'location.country', 'location.formattedAddress',
       'location.address', 'location.city', 'location.state',
       'location.crossStreet', 'location.neighborhood', 'location.postalCode'],
      dtype='object')

In [40]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [79]:
filtered_columns = ['name','categories','location.lat', 'location.lng']
veterinary =veterinary.loc[:, filtered_columns]
veterinary.head()

,name,categories,location.lat,location.lng
0,Veterinaria San Roque,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",4.708541,-74.071816
1,Veterinaria dr david bernal,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",4.708601,-74.075333
2,Veterinaria PRE PAY Grooming Club,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",4.722691,-74.050319
3,Súper Waw Clinica veterinaria,"[{'id': '4d954af4a243a5684765b473', 'name': 'V...",4.710816,-74.074530
4,Veterinaria animal work,"[{'id': '4bf58dd8d48988d100951735', 'name': 'P...",4.716472,-74.065061


In [80]:
veterinary['categories'] = veterinary.apply(get_category_type, axis=1)
# clean columns
veterinary.columns = [col.split(".")[-1] for col in veterinary.columns]
veterinary.head(25)

,name,categories,lat,lng
0,Veterinaria San Roque,Pet Store,4.708541,-74.071816
1,Veterinaria dr david bernal,Veterinarian,4.708601,-74.075333
2,Veterinaria PRE PAY Grooming Club,Hospital,4.722691,-74.050319
3,Súper Waw Clinica veterinaria,Veterinarian,4.710816,-74.074530
4,Veterinaria animal work,Pet Store,4.716472,-74.065061
5,CRE Centro de Resonancia Especializada Veterin...,Veterinarian,4.710770,-74.074974
6,Veterinaria,None,4.729281,-74.068234
7,Veterinaria Dog Land,Veterinarian,4.700618,-74.079267
8,Veterinaria Prado,Pet Service,4.715114,-74.058563
9,Clinica Oftalmologica Veterinaria Audrey Calderon,Pet Service,4.706380,-74.078500


### Top categories pet service

In [81]:
# Top 10 Categories pet services
a=pd.Series(veterinary.categories)
a.value_counts()[:10]

Veterinarian        23
Pet Store           10
Pet Service          6
Acupuncturist        4
Animal Shelter       3
Business Service     1
Hospital             1
Name: categories, dtype: int64

## K-Means Clustering Approach

In [83]:
# Using K-Means to cluster pet services into 3 clusters
veterinary_clustering = veterinary.drop('name', 1)
veterinary_clustering = veterinary_clustering.drop('categories', 1)
kmeans = KMeans(n_clusters=3, random_state=0).fit(veterinary_clustering)
kmeans.labels_

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 2, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 1, 2, 0, 0, 0, 2, 1, 1, 0, 1, 0, 1, 1, 1, 2, 2, 0, 2,
       2, 0, 1, 1, 0, 0], dtype=int32)

In [84]:
veterinary.head()

,name,categories,lat,lng
0,Veterinaria San Roque,Pet Store,4.708541,-74.071816
1,Veterinaria dr david bernal,Veterinarian,4.708601,-74.075333
2,Veterinaria PRE PAY Grooming Club,Hospital,4.722691,-74.050319
3,Súper Waw Clinica veterinaria,Veterinarian,4.710816,-74.074530
4,Veterinaria animal work,Pet Store,4.716472,-74.065061


In [91]:
## veterinary.insert(0, 'Cluster Labels', kmeans.labels_)

## veterinary_merged =df_2.iloc[:16,:]

## veterinaryh_merged = veterinary_merged.join(veterinary.set_index('name'), on='name')

veterinary.head()# check the last columns!

,Cluster Labels,name,categories,lat,lng
0,0,Veterinaria San Roque,Pet Store,4.708541,-74.071816
1,0,Veterinaria dr david bernal,Veterinarian,4.708601,-74.075333
2,1,Veterinaria PRE PAY Grooming Club,Hospital,4.722691,-74.050319
3,0,Súper Waw Clinica veterinaria,Veterinarian,4.710816,-74.074530
4,1,Veterinaria animal work,Pet Store,4.716472,-74.065061


## Map of Clusters

In [93]:
kclusters = 10

In [100]:
# create map
latitude = 4.710113
longitude = -74.072036
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]
print(rainbow)
# add markers to the map

markers_colors = []
for lat, lon, nei , cluster in zip(veterinary['lat'], 
                                   veterinary['lng'], 
                                   veterinary['name'], 
                                   veterinary['Cluster Labels']):
    label = folium.Popup(str(nei) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

['#8000ff', '#4856fb', '#10a2f0', '#2adddd', '#62fbc4', '#9cfba4', '#d4dd80', '#ffa256', '#ff562c', '#ff0000']
